In [37]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

city = 'gdansk'
url = 'https://www.olx.pl/nieruchomosci/mieszkania/sprzedaz/{}/?page='.format(city)

columns = ['Title', 'Price', 'Sq_Footage', 'Price_Per_Meter']
df = pd.DataFrame(columns=columns)
cards = None

In [38]:
sq_footages = []

pages = 1
for page_index in range(pages):
	next_url = url + str(page_index + 1)
	response = requests.get(url)

	if response.status_code == 200:
		soup = BeautifulSoup(response.text, 'html.parser')
		cards = soup.find_all('div', {'data-cy': 'l-card'})

	for card in cards:
		title = card.find('h6').text
		price = card.find('p').text
		price = int(price[:price.index('z')].replace(' ', ''))
		sq_footage_data = card.find('span', {'class': 'css-643j0o'}).text
		sq_footage = float(sq_footage_data[:sq_footage_data.index('m') - 1].replace(',', '.'))
		price_per_meter = float(sq_footage_data[sq_footage_data.index('-') + 1 : sq_footage_data.index('z') - 1])

		df.loc[len(df)] = [title, price, sq_footage, price_per_meter]

In [39]:
df

,Title,Price,Sq_Footage,Price_Per_Meter
0,"Mieszkanie 3-pokojowe 66,96 m². Gdańsk, Przymorze",737500,66.96,11014.04
1,72 m2 Mieszkanie w samym sercu Wrzeszcza,990000,71.00,13943.66
2,"Gdańsk Morena 2-pok 49m.kw, umeblowane, osiedl...",660000,49.00,13469.39
3,Okazja! Urokliwe Mieszkanie z Potencjałem w Śr...,578000,36.00,16055.56
4,Lokal + marina 40m2 Sobieszewo / Gdańsk SolMarina,999000,39.86,25062.72
5,"Wyposażone mieszkanie, 2 m. parkingowe W CENIE !!",640000,42.93,14907.99
6,Dni Otwarte | Widok Na Morze | Centralpark,689000,50.21,13722.37
7,"Wykończone mieszkanie na Jasieniu, od ręki",489000,36.00,13583.33
8,"Apartament inwestycyjny Śląska 12, k/Skm i Ug!",429524,25.72,16700.00
9,"Piękne dwa pokoje, gotowe, przy plaży :)",698000,41.07,16995.37
